### Airbnb Scraping

In [7]:
from selenium import webdriver  #importar Selenium webdriver y tener acceso a ella
import os
import time
from bs4 import BeautifulSoup
import pandas as pd
import warnings  #Ignorar los mensajes "warnings" 
warnings.filterwarnings('ignore')


# ---------------------------------->2) Listas <---------------------------------
title=[]
rating=[]
city=[]
href=[]
price=[]
tipe=[]
host=[]
rooms=[]
beds=[]
bathrooms=[]
days=[]
days_2=[]

#Set para colección de datos
collection=[]


#------------------------------->3)Creando clases y métodos<-------------------


class Airbnb(webdriver.Chrome):
    def __init__(self,driver_path=os.pathsep + r'C:\SeleniumDrivers', teardown=False):
        self.driver_path=driver_path
        self.teardown=teardown
        os.environ["PATH"] += self.driver_path
        options = webdriver.ChromeOptions()
        options.add_experimental_option('excludeSwitches', ['enable-logging'])    #Evitar adverntencias del desarrollo   
        super(Airbnb,self).__init__(options=options)
        self.implicitly_wait(10) # Espera hasta que se carguen los elementos
        self.maximize_window() # maximizar windows 
        
        
        
    def __exit__(self,exc_type,exc_value,exc_tb):
        if self.teardown:
            self.quit()

    #// Entrada de URL 
    def land_firts_page(self):
        self.get("https://www.airbnb.com.co/s/bogota/homes?tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&flexible_trip_dates%5B%5D=december&flexible_trip_dates%5B%5D=november&flexible_trip_lengths%5B%5D=weekend_trip&date_picker_type=calendar&source=structured_search_input_header&search_type=search_query")
        
        
     #--------------> Cookie <---------------------#
    def cookie(self):
        cookie_element=self.find_element_by_css_selector('button._1932igt4')
        try:
            cookie_element.click()
        except:
            pass 
        
    #----------------> Elementos y paginación<-------------------
    
    def element_parser(self):
        for j in range(1,15):
            print("ejecutando sin problemas, espere mientras se cumple el scraping de la página")
            for i in range(1,20):
                element = self.find_element_by_xpath("(//div[@class='_gig1e7'])["+str(i)  +"]")
                element.click()
                time.sleep(2)
                # Hacemos foco sobre la nueva pestaña
                self.switch_to.window(self.window_handles[1])
                # --- Ahora estamos trabajando en la URL-secundaria
                time.sleep(4) 
                
                #----------------------Elementos de la página--------------------------
                # title
                try:
                    title_house=self.find_element_by_xpath('//*[@id="site-content"]/div/div[1]/div[1]/div[1]/div/div/div/div/section/div[1]/span/h1')
                    title.append(title_house.text)
                except:
                    title.append(None)
                
                # Rating
                try:
                    rating_house=self.find_element_by_css_selector("span._1ne5r4rt")
                    rating.append(rating_house.text)
                except:
                    rating.append(None)

                # city
                try:

                    city_house=self.find_element_by_css_selector("span._pbq7fmm")
                    city.append(city_house.text)
                except:
                    city.append(None)

                # Url
                try:
                    href_house=self.current_url
                    href.append(href_house)
                except:
                    href.append(None)

                # price/night
                try:
                    price_house=self.find_element_by_xpath('//*[@id="site-content"]/div/div[1]/div[3]/div/div[2]/div/div/div[1]/div/div/div/div/div/div/div/div[1]/div[1]/div/div/div/span[1]')
                    price.append(price_house.text)
                except:
                    price.append(None)

                # tipe
                try:
                    type_house=self.find_element_by_css_selector("h2._14i3z6h")
                    tipe.append(type_house.text)
                except:
                    tipe.append(None)


                # host- titulo del anfitriòn
                try:
                    host_house=self.find_element_by_xpath('//*[@id="site-content"]/div[1]/div/div/section/div/div/div/div[1]/ol/li[1]/span')
                    host.append(host_house.text)
                except:
                    host.append(None)

                # Rooms--Cuartos
                try:
                    rooms_house=self.find_element_by_xpath('//*[@id="site-content"]/div[1]/div/div/section/div/div/div/div[1]/ol/li[2]/span[2]')
                    rooms.append(rooms_house.text)
                except:
                    rooms.append(rooms_house.text)


                # bathrooms --baños
                try:
                    beds_house=self.find_element_by_xpath('//*[@id="site-content"]/div[1]/div/div/section/div/div/div/div[1]/ol/li[3]/span[2]')
                    beds.append(beds_house.text)
                except:
                    beds.append(beds_house.text)

                # beds --camas
                try:
                    bathrooms_houses=self.find_element_by_xpath('//*[@id="site-content"]/div[1]/div/div/section/div/div/div/div[1]/ol/li[4]/span[2]')
                    bathrooms.append(bathrooms_houses.text)
                except:
                    bathrooms.append(None)
                    
                  #Días Disponibles
                
                soup = BeautifulSoup(self.driver_path, 'lxml')
                try:       
                    days_air= soup.find_all("div",{"data-is-day-blocked":"false"})
                    days.append([(i.text) for i in days_air])
                except:
                    days.append(None)
        
            
            #------------------> Foco de la página <-----------------------------   
            
                time.sleep(3)
                # Cerrar la nueva pestaña de URL-secundaria
                self.close()

                # Cambiar el foco, para volver a la URL-principal
                self.switch_to.window(self.window_handles[0])
                time.sleep(3) 
                
            try: 
                next_page=self.find_element_by_xpath('//a[@aria-label="Siguiente"]')
                self.execute_script("arguments[0].click();", next_page) 
            except:
                print("No se encuentran mas páginas")
                
        time.sleep(5)         
        a={"title":title,"host":host,"city":city,"price":price,"rating":rating,"rooms":rooms,"bathrooms":bathrooms,"beds":beds,"tipe":tipe,"href":href,"days":days}
        df = pd.DataFrame.from_dict(a, orient='index')
        df = df.transpose()
        print(df)
        #Acá se guarda el archivo con el nombre establecido
        df.to_excel("Airbnb2.xlsx", index=False)       
            


In [8]:
with Airbnb() as bot:
    bot.land_firts_page()
    time.sleep(2)
    bot.cookie()
    time.sleep(2)
    bot.element_parser()
        
    
     

ejecutando sin problemas, espere mientras se cumple el scraping de la página
                                                title         host  \
0                      MOUNTAIN VIEW MODERN APARTMENT  2 huéspedes   
1              Apto Semi Amoblado Excelente Ubicacion  2 huéspedes   
2                         Maravilloso Loft entero 302  2 huéspedes   
3    DESCUENTO % Tu Bioseguridad es nuestra Prioridad  2 huéspedes   
4   Casa de los Colibrís 2: Loft apartaestudio entero  2 huéspedes   
5                        APARTA SUITE VITRA CHAPINERO  3 huéspedes   
6        Relajante apartamento en el centro histórico  3 huéspedes   
7      COMODO *CORFERIAS * CENTRAL* AIRPORT* EMBASSY*  2 huéspedes   
8    Hermoso Apartaestudio con vista al parque Alcalá  2 huéspedes   
9        Nuevo Apartaestudio- Gran vista Cra 7 Zona G  2 huéspedes   
10                         Awesome loft in Bogotá 204  2 huéspedes   
11  Apto nuevo Chapinero, excelente ubicación y ac...  3 huéspedes   
12           